In [1]:
import pickle
import matplotlib.pyplot as plt
from ctypes import *
import numpy as np
import base64
import matplotlib
from collections import Counter

In [2]:
from lib.proto.protocol_pb2 import *
from google.protobuf.json_format import MessageToJson, ParseDict, MessageToDict
from lib.proto.protocol_pb2 import DatumTypeID, LinkID, TalkerID
import lib.proto.protocol_pb2 as protocol
from lib.comms_lib.crctabgen import crc16
# from link import cobs_decode

In [3]:
segments = pickle.load(open("./format_v2_test.pickle", 'rb'))
segments, ack = segments[:-1], segments[-1]
crc_xor = ack['log_crc16']

In [4]:
print(segments[0].keys())

dict_keys(['start_address', 'length', 'data', 'segment_crc16'])


In [5]:
xor = 0
for s in segments:
    xor ^= s['segment_crc16']
if (xor == crc_xor):
    print("Data valid!")

Data valid!


In [6]:
LOGGER_COBS_ESC =0xAA
LOGGER_COBS_ESC_AA =0x01
LOGGER_COBS_ESC_FF =0x02
def cobs_decode(data: bytes) -> bytes:
    dout = bytes()
    diter = iter(data)
    while (d := next(diter, None)) is not None:
        if (d == LOGGER_COBS_ESC):
            dout += bytes([LOGGER_COBS_ESC if next(diter, None)
                          == LOGGER_COBS_ESC_AA else 0xFF])
        else:
            dout += bytes([d])
    return dout

In [7]:
# TODO: Change the escape byte!!! 00 is too common...
databuffer = bytes()
for s in segments:
    databuffer += base64.b64decode(s['data'])
# data = cobs_decode(databuffer)
# print(len(data), len(databuffer))

In [8]:
frames = databuffer.split(b'\xFF')
frames_dec = [e for e in [cobs_decode(f) for f in frames] if len(e)]
print(Counter([len(f) for f in frames_dec]))
# assert len({len(f) for f in frames_dec}) == 1
# len(frames_dec)

Counter({74: 3837})


In [9]:
# typedef struct log_data_t {
#     // Sensor data
#     uint32_t lps_press_raw;
#     uint16_t lis_mag_raw[3];
#     uint16_t lsm_acc_raw[3];
#     uint16_t lsm_gyr_raw[3];
#     float adxl_acc_raw[3]; // TYPE TBD!!!
#     // Processed data
#     float orientation_quat[4];
#     float filtered_altitude_m;
#     // From GPS
#     float gps_lat;
#     float gps_lon;
#     float gps_alt;
#     uint8_t flags;
# } log_data_t;

class LogDataT(Structure):
    _fields_ = [
        ("lps_press_raw", c_uint32),
        ("lis_mag_raw", c_int16*3),
        ("lsm_acc_raw", c_int16*3),
        ("lsm_gyr_raw", c_int16*3),
        ("adxl_acc_raw", c_int16*3),
        ("orientation_quat", c_float*4),
        ("filtered_altitude_m", c_float),
        ("gps_lat", c_float),
        ("gps_lon", c_float),
        ("gps_alt", c_float),
        ("flags", c_uint8),
    ]
    
assert (sizeof(LogDataT)+2+8) == len(frames_dec[0])
print(sizeof(LogDataT))

64


In [10]:
frames_dec.sort(key=lambda b: int.from_bytes(b[0:8], 'little'))


parsed_frames = [LogDataT.from_buffer_copy(b[2+8:]) for b in frames_dec]

In [11]:
time_seconds = np.array([int.from_bytes(b[0:8], 'little') for b in frames_dec]) * 1e-6
%matplotlib qt

In [12]:
# acc_x = np.array([f.lsm_acc_raw[2] for f in parsed_frames])
# plt.plot(time, acc_x)
# plt.show()
# plt.plot(np.array([f.TIMESTAMP for f in parsed_frames]))
# plt.show()
# parsed_frames[0].TIMESTAMP, parsed_frames[1].TIMESTAMP, parsed_frames[2].TIMESTAMP

In [13]:
# All in units of Gs
adxl_xs = np.array([f.adxl_acc_raw[0] * 49 / 1000.0 for f in parsed_frames])
adxl_ys = np.array([f.adxl_acc_raw[1] * 49 / 1000.0 for f in parsed_frames])
adxl_zs = np.array([f.adxl_acc_raw[2] * 49 / 1000.0 for f in parsed_frames])

In [14]:
# All in units of Gs
lsm_acc_xs = np.array(
    [f.lsm_acc_raw[0] * 0.488 / 1000.0 for f in parsed_frames])
lsm_acc_ys = np.array(
    [f.lsm_acc_raw[1] * 0.488 / 1000.0 for f in parsed_frames])
lsm_acc_zs = np.array(
    [f.lsm_acc_raw[2] * 0.488 / 1000.0 for f in parsed_frames])

In [15]:
# All in units of degrees/second
lsm_gyr_xs = np.array(
    [f.lsm_gyr_raw[0] * 70.0 / 1000.0 for f in parsed_frames])
lsm_gyr_ys = np.array(
    [f.lsm_gyr_raw[1] * 70.0 / 1000.0 for f in parsed_frames])
lsm_gyr_zs = np.array(
    [f.lsm_gyr_raw[2] * 70.0 / 1000.0 for f in parsed_frames])

In [16]:
# All in units of gauss
lis_mag_xs = np.array([f.lis_mag_raw[0]/6842.0 for f in parsed_frames])
lis_mag_ys = np.array([f.lis_mag_raw[1]/6842.0 for f in parsed_frames])
lis_mag_zs = np.array([f.lis_mag_raw[2]/6842.0 for f in parsed_frames])

In [17]:
lats = np.array([f.gps_lat for f in parsed_frames])
lons = np.array([f.gps_lon for f in parsed_frames])
g_alts = np.array([f.gps_alt for f in parsed_frames])

In [18]:
alts = np.array([f.lps_press_raw for f in parsed_frames])

In [19]:
fig, ((ax1), (ax2), (ax3)) = plt.subplots(3, 1)

# # tot_gyr = np.sqrt(lsm_gyr_xs**2 + lsm_gyr_ys**2 + lsm_gyr_zs**2)



ax1.plot(time_seconds, lsm_acc_xs)

# ax1.plot(time_seconds, adxl_xs)

ax2.plot(time_seconds, lsm_acc_ys)

# ax2.plot(time_seconds, adxl_ys)

ax3.plot(time_seconds, lsm_acc_zs)
# ax3.plot(time_seconds, adxl_zs)

# # # ax4.plot(time_seconds, tot_gyr)


fig.show()

In [20]:
##### FORWARD AXIS: X!

In [21]:
# # plt.plot(time_seconds, -lsm_acc_xs)

# Mrocket = 1.0 # Kg
# fwd_force_n = -adxl_xs*9.81*Mrocket
# plt.plot(time_seconds, fwd_force_n)